In [2]:
import tensorflow as tf

In [3]:
X = tf.placeholder(tf.float32, [None,64,64,1])
Y = tf.placeholder(tf.float32, [None, 2350])
keep_prob = tf.placeholder(tf.float32)

In [4]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
L1 = tf.nn.relu(L1)

L1 =  tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [11]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)

L2 =  tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L2)

Tensor("MaxPool_7:0", shape=(?, 16, 16, 64), dtype=float32)


In [12]:
W3 = tf.Variable(tf.random_normal([5, 5, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(L2,W3, strides=[1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)

L3 =  tf.nn.max_pool(L3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L3)

Tensor("MaxPool_8:0", shape=(?, 8, 8, 128), dtype=float32)


In [10]:
W4 = tf.Variable(tf.random_normal([5, 5, 64, 256], stddev=0.01))

L4 = tf.nn.conv2d(L3,W4, strides=[1,1,1,1], padding = 'SAME')
L4 = tf.nn.relu(L3)

L4 =  tf.nn.max_pool(L4, ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
print(L4)

Tensor("MaxPool_6:0", shape=(?, 8, 8, 64), dtype=float32)


In [14]:
W5 = tf.Variable(tf.random_normal([8*8*256, 1024], stddev=0.01))
L5 = tf.reshape(L4,[-1,8*8*256])
L5 = tf.matmul(L5, W5)
L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5,keep_prob)

In [17]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
model = tf.matmul(L5, W6)

In [22]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)